In [2]:
import pandas as pd
df = pd.read_csv('../data/corpus_check.csv')

In [3]:
df

,siren,legal_name,id,url_article,corpus
0,419838529,IPSEN,8,http://www.boursier.com/actions/actualites/new...,Ipsen lorgne les peptides de PeptiMimesis\n
1,419838529,IPSEN,2894,http://www.boursier.com/actions/actualites/new...,Ipsen : accord important avec Probi\n \n\npubl...
2,419838529,IPSEN,3057,http://www.boursier.com/actions/actualites/new...,La médecine générale d'Ipsen en panne au 1er t...
3,419838529,IPSEN,4208,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
4,419838529,IPSEN,5284,http://www.cerclefinance.com/default.asp?pub=v...,"Bourse en ligne : Information boursiere, Econo..."
...,...,...,...,...,...
495,395030844,SANOFI,41863,https://www.lemondedudroit.fr/deals/57157-clif...,Clifford Chance conseil de banques sur l'émiss...
496,395030844,SANOFI,42528,https://www.lesechos.fr/industrie-services/pha...,Sanofi fragilisé par le recul de son activité ...
497,395030844,SANOFI,43721,https://www.lesechos.fr/industrie-services/pha...,Sanofi prépare son entrée sur le marché de l'a...
498,395030844,SANOFI,43756,https://www.mypharma-editions.com/sanofi-le-do...,» Sanofi : le Docteur Jacques Volckmann nommé ...
